In [1]:
import numpy as np
import math


In [2]:
def area_flow(Dia):
    A_cross = math.pi * (Dia / 2) ** 2
    return A_cross



In [3]:
def get_LMTD_counter(T_hot_in, T_cold_out, T_hot_out, T_cold_in):
    # Log Mean Temperature Difference (LMTD) for counter-flow
    delta_T1 = T_hot_in - T_cold_out
    delta_T2 = T_hot_out - T_cold_in
    
    if abs(delta_T1 - delta_T2) < 1e-10:
        LMTD = delta_T1  # Avoid division by zero when delta_T1 = delta_T2
    else:
        LMTD = (delta_T1 - delta_T2) / math.log(delta_T1 / delta_T2)

    return LMTD

In [4]:
def get_mass_flow(rho, velocity,A_cross):
    m_dot = rho * velocity * A_cross
    return m_dot


In [5]:
def get_Q(mass_flow, cp, temp_diff):
    Q = mass_flow*cp*temp_diff
    return Q

In [6]:
def get_h(cp,mu,m_dot,km,dia,kf):
    Prandtl_number = cp*mu/km 
    reynolds = (4*m_dot)/(math.pi*dia*mu)
    h = 0.023 * reynolds**0.8 * Prandtl_number**0.4 * (kf/dia)

    return h



In [7]:
def get_LMTD_parra(T_hot_in, T_cold_out, T_hot_out, T_cold_in):
    # Log Mean Temperature Difference (LMTD) for counter-flow
    delta_T1 = T_hot_in - T_cold_in
    delta_T2 = T_hot_out - T_cold_out
    
    if abs(delta_T2 - delta_T1) < 1e-10:
        LMTD = delta_T2  # Avoid division by zero when delta_T1 = delta_T2
    else:
        LMTD = (delta_T2 - delta_T1) / math.log(delta_T2 / delta_T1)

    return LMTD

In [8]:
#Find Qc for testing toy model

Cp_c = 4180 #J/kg K
m_dot_c = 0.2 #kg/s
T_co = 60 #c
T_ci = 25 #c

dia = 0.008 #m thin wall so dia outter is same as d inner

Qc = get_Q(m_dot_c,Cp_c,(T_co - T_ci))
print(Qc) #w

#Lets find T_ho
Cp_h = 4310 #J/kg K
m_dot_h = 0.3 #kg/s
T_hi = 140 #c

T_ho = T_hi - Qc/(m_dot_h * Cp_h)
print(T_ho)

LMTD_para = get_LMTD_parra(T_hi,T_co,T_ho,T_ci)
print(LMTD_para)

U = 550 #w/m2 K
surface_area = Qc/(U * LMTD_para) #m2
print(surface_area)

Length =  surface_area/(math.pi*dia)
print(Length)

29260.0
117.37045630317093
82.87219381751132
0.6419523551789764
25.542472639054544


In [9]:
#Current purifier

Cp = 5188 #J/kg K
v_dot = 3.5 #cuft
T_co = 80 #k
T_hi = 295 #k
T_ho = 273 #k guess

rho_25 = 0.1607 #kg/m3
rho_cold = 0.622 #kg/m3

#unit conversion
L_to_m = 0.001 #m3
cuft_to_l = 28.3168 #l
cuft_to_m = cuft_to_l*L_to_m
in_to_m = 0.0254

rho_25 = rho_25*cuft_to_m
m_dot = v_dot*rho_25/60 #kg/s

#==-=--=-=-=-=-=-==-=-=-=-=--=-=-=-=-==-==-=-=-=-=-==-=-

Qh = abs(get_Q(m_dot,Cp,(T_ho - T_hi)))

T_ci = T_co - Qh/(m_dot * Cp)
print(f"Inside cold temp is: {T_ci}")

LMTD_counter = get_LMTD_counter(T_hi,T_co,T_ho,T_ci)
print(f"Log mean temp is: {LMTD_para}")

#Getting h
mu = 15e-6 #pa
kf = .1
km = 15

#We have two sections bed and upper
dia1 = .5*in_to_m
h1 = get_h(Cp,mu,m_dot,km, dia1,kf)
dia2 = (3/4)*in_to_m
h2 = get_h(Cp,mu,m_dot,km, dia2,kf)
U = (1/(h1*math.pi*dia1))+(1/(h2*math.pi*dia2))+math.log(dia2/dia1)/(2*math.pi*km) #w/m2 K
print(U)
surface_area = Qh/(U * LMTD_counter) #m2
print(surface_area)

Length =  surface_area/(math.pi*dia)
print(Length) #m

#We know the length is 28" 
L = 28*in_to_m
surface_area = math.pi*(dia2-dia1)*L
Q = U*surface_area*LMTD_counter
print(f"Q is {Q}")

Qc = get_Q(m_dot,Cp,(T_hi - T_co))
print(Qc)
print(Qc/Q)

Inside cold temp is: 58.0
Log mean temp is: 82.87219381751132
6.8132852654802045
0.02068256794287754
0.8229332309857333
Q is 20.783102459486464
296.08422646245333
14.246392088939803


In [10]:
def get_partial_pressure(mole_fraction, P_sat):
    partial_pressure = mole_fraction*P_sat/100
    return partial_pressure

In [37]:
mole_fraction_N2 = 0.00078*100 # % precentage
mole_fraction_O2 = 0.00022*100 # precentage

Temp = 80 #K
P_sat_N2 = 1 # bar
P_sat_O2 = .2 #bar

operating_pressure = 0.689476 #bar
flow_rate = 9#4 #nm3/h equaled to ~60 l per min 9 is 150slm



partial_N2 = get_partial_pressure(mole_fraction_N2,P_sat_N2)
partial_O2 = get_partial_pressure(mole_fraction_O2,P_sat_O2)
print(f"partial P N2, {partial_N2} and partial P O2, {partial_O2}.")
partial_air = partial_N2 + partial_O2
print(f"partial P air: {partial_air}")

mole_fraction_air = (partial_air/operating_pressure)
print(f"Mole fraction of air is {mole_fraction_air}%")

t = 24*7 #hours
nitrogen_adsorbed = mole_fraction_air*flow_rate*t
print(f"Nitrogen needed to be adsorbed is: {nitrogen_adsorbed}")
v_a = nitrogen_adsorbed*1000000 #cc
energy_of_adsorption = 300.2 #k
c = math.exp(energy_of_adsorption/Temp)
vm = 379 #might not be true? units g cite:Juan de Dios Lopez-Gonzalez,! Frank G . Carpenter,! and Victor R. Deitz
constant = (1+c*(operating_pressure/P_sat_N2))/(c*(operating_pressure/P_sat_N2))
factor_of_safety = 3
mass_of_adsorpant = (v_a/vm)*constant
print(f"The mass of the adsorpant is {mass_of_adsorpant} grams, with FS of {factor_of_safety} it is {mass_of_adsorpant*factor_of_safety}.")


#Sizing of bed

g_to_kg = 0.001
density_carbon = 0.44 # g/cc This is the bulk density for GAC
volume_carbon = mass_of_adsorpant/(density_carbon)
cc_to_cuft = 3.5315e-5 # cuft
volume_carbon = volume_carbon*cc_to_cuft
print(f"The volume needed is {volume_carbon} cuft")

R = 7 #L/D this is based on the velocity of the gas!
cuft_to_cuin = 1728
Dia = ((4*volume_carbon*cuft_to_cuin)/(math.pi*R))**(1/3)
L = ((4*volume_carbon*cuft_to_cuin*R**2)/(math.pi))**(1/3) #inches

print(f"The diameter of the bed should be {Dia} and Lenght {L}")


partial P N2, 0.00078 and partial P O2, 4.4000000000000006e-05.
partial P air: 0.000824
Mole fraction of air is 0.0011951104897052255%
Nitrogen needed to be adsorbed is: 1.807007060434301
The mass of the adsorpant is 4930.051287264066 grams, with FS of 3 it is 14790.153861792198.
The volume needed is 0.3956926391130239 cuft
The diameter of the bed should be 4.9915787752123775 and Lenght 34.94105142648664


In [12]:

def adsorbent_mass_required(
    flow_rate_slm=60,          # total flow, standard liters per minute
    inlet_purity=0.99,         # helium purity (fraction)
    target_purity=0.99999,     # desired helium purity (fraction)
    ads_capacity=0.60,         # g of gas adsorbed per g adsorbent (from CTC value)
    molar_mass_air=28.97,      # g/mol
    duration_hr=24,            # operating time in hours
    safety_factor=3.0          # safety factor
):

    
    # Standard molar volume (L/mol at STP)
    V_molar = 22.414  

    # Flow of impurity (air) in L/min
    air_inlet_frac = 1 - inlet_purity
    air_outlet_frac = 1 - target_purity
    air_removed_frac = air_inlet_frac - air_outlet_frac
    air_removed_flow = flow_rate_slm * air_removed_frac

    # Convert to mol/min and g/min
    mol_air_removed = air_removed_flow / V_molar
    g_air_removed_per_min = mol_air_removed * molar_mass_air

    # Adsorbent mass needed per minute and total
    g_adsorbent_per_min = g_air_removed_per_min / ads_capacity
    total_adsorbent_g = g_adsorbent_per_min * 60 * duration_hr

    return {
        "air_removed_flow_Lmin": air_removed_flow,
        "air_removed_g_per_min": g_air_removed_per_min,
        "adsorbent_g_per_min": g_adsorbent_per_min,
        "total_adsorbent_g": total_adsorbent_g,
        "total_adsorbent_kg": total_adsorbent_g / 1000
    }



result = adsorbent_mass_required(
    flow_rate_slm=60,
    inlet_purity=0.99,
    target_purity=0.99999,
    ads_capacity=0.70,  # from 60 CTC
    duration_hr=200,
    safety_factor=1.0
)

print("=== Helium Purification Adsorbent Calculation ===")
print(f"Air removed flow: {result['air_removed_flow_Lmin']:.4f} L/min")
print(f"Air removed: {result['air_removed_g_per_min']:.4f} g/min")
print(f"Adsorbent required per minute: {result['adsorbent_g_per_min']:.4f} g")
print(f"Total adsorbent required: {result['total_adsorbent_kg']:.3f} kg")

g_to_kg = 0.001
density_carbon = 0.44 # g/cc This is the bulk density for GAC
volume_carbon = result['total_adsorbent_g']/(density_carbon)
cc_to_cuft = 3.5315e-5 # cuft
volume_carbon = volume_carbon*cc_to_cuft
print(f"The volume needed is {volume_carbon} cuft")


=== Helium Purification Adsorbent Calculation ===
Air removed flow: 0.5994 L/min
Air removed: 0.7747 g/min
Adsorbent required per minute: 1.1067 g
Total adsorbent required: 13.281 kg
The volume needed is 1.0659469869886573 cuft


In [13]:
#How much adsorbant do we need?
break_through_time = 3 #days
day_to_sec = 86400 #s
g_to_kg = 0.001
Mass_flow_rate_N2 = m_dot*mole_fraction_N2
print(Mass_flow_rate_N2)
Nitrogen_required_to_be_adsorbed = break_through_time*day_to_sec*Mass_flow_rate_N2 #gN2
print(Nitrogen_required_to_be_adsorbed*g_to_kg)

uptake_carbon = 195.2 #cc/kgC
density_n2 = 794 #kg/m3 at 80k
density_carbon = 0.44 # g/cc
m3_to_cm3 = 1/1000000

#Using the above calculation
mass_of_carbon = (Nitrogen_required_to_be_adsorbed*g_to_kg)/(density_n2*uptake_carbon*m3_to_cm3)
print(f"The mass needed is: {mass_of_carbon} kg")

volume_carbon = mass_of_carbon/(density_carbon/g_to_kg)
cc_to_cuft = 3.5315e-5 # cuft
volume_carbon = volume_carbon*cc_to_cuft
print(f"with a volume of {volume_carbon} cuft")

#Sizing of bed
R = 10 #L/D this is based on the velocity of the gas!
cuft_to_cuin = 1728
Dia = (4*volume_carbon*cuft_to_cuin/math.pi*R)**(1/3)
L = ((4*volume_carbon*cuft_to_cuin)/(math.pi*Dia**2)) #inches
print(Dia,L) #inches


2.0704819407999996e-05
0.005366689190553599
The mass needed is: 0.03462630325903291 kg
with a volume of 2.779154317256244e-09 cuft
0.03939630639312785 0.003939630639312784


In [60]:
#Jlab code

def polanyi_potential_theory(R,T,Ps,p):

    specific_adsorbent_capacity = R*T*math.log(Ps/p)
    return specific_adsorbent_capacity

def get_mass_carbon(rho,adsorbed,capacity):
    m3_to_cm3 = 1/1000000
    g_to_kg = 1/1000
    mass = (adsorbed*g_to_kg)/(rho * capacity * m3_to_cm3)
    return mass
def get_volume_carbon(mass):
    g_to_kg = 1/1000
    cm3_to_cuft = 3.53147e-5
    density_carbon = 0.44 #g/cc
    vol = mass/(density_carbon*g_to_kg)
    vol = vol*cm3_to_cuft
    return vol


def get_g_per_s_flow(SLM = 60):
    # Constants
    RHO_He_STP = 0.1786          # helium density at STP [g/L]
    SECONDS_PER_MINUTE = 60

    # Convert SLM to g/s
    mass_flow_g_per_s = SLM * RHO_He_STP / SECONDS_PER_MINUTE

    print(f"{SLM} SLM of helium = {mass_flow_g_per_s:.4f} g/s")
    return mass_flow_g_per_s


#GIVENS
R = 0.2968 #kj/kg-k
T = 80 #K
mole_fraction_helium = 0.999
mole_fraction_N2_Jlab = 0.001

Ps_N2 = 137.0 #kpa
ppm_N2 = mole_fraction_N2_Jlab*1e6 #pmm
P_total = 41.3685 #kpa


# #get partial_pressure N2
Partial_N2 = ppm_N2*P_total/1e6 #kPa

specific_adsorbent = polanyi_potential_theory(R,T,Ps_N2,Partial_N2)

#print(f"The specific adsorbent capacity is: {specific_adsorbent} kJ/kg")
KJ_to_cal = 239.006
kg_per_mol_N2 = 0.0280134

specific_adsorbent_cal_per_mol = specific_adsorbent * KJ_to_cal * kg_per_mol_N2


print(f"The specific adsorbent capacity is: {specific_adsorbent_cal_per_mol} cal/mol")


#Next find mass concentration of nitrogen in helium

mol_he = 4.0026 #Kg/kmol
mass_helium_mix = mole_fraction_helium*mol_he #kg

mol_N2 = 28.0134 #Kg/kmol

mass_N2_mix = mole_fraction_N2_Jlab*mol_N2 #kg

Total_mass_mix = mass_helium_mix + mass_N2_mix
percent_mass_n2 = mass_N2_mix/Total_mass_mix

#Find Nitrogen Partial Flow
Mass_flow_rate_He_Jlab = get_g_per_s_flow(150) #g/s
Mass_flow_rate_N2_JLab = Mass_flow_rate_He_Jlab*percent_mass_n2 #g/s
#Now we find the specific weight of adsorbent
break_through_time = 4
 #Days
day_to_sec = 86400
N2_needed_adsorbed = break_through_time*Mass_flow_rate_N2_JLab*day_to_sec
print(f"We need to adsorb {N2_needed_adsorbed/1000} kg of N2 to run {break_through_time} days")

capacity_carbon_N2 = 200 #cc/kgC Approx #300
density_n2 = 794 #kg/m3

mass_of_carbon = get_mass_carbon(density_n2,N2_needed_adsorbed,capacity_carbon_N2)
print(f"We need {mass_of_carbon} kg of carbon for the bed")
volume_carbon = get_volume_carbon(mass_of_carbon)
print(f"This relates to a volume of {volume_carbon} cuft")

# R = 8 #L/D this is based on the velocity of the gas!
# cuft_to_cuin = 1728
# Dia = ((4*volume_carbon*cuft_to_cuin)/(math.pi*R))**(1/3)
# L = ((4*volume_carbon*cuft_to_cuin*R**2)/(math.pi))**(1/3) #inches
Dia = 6.36 #inch
L = (volume_carbon * cuft_to_cuin)/((Dia/2)**2 * math.pi)

print(f"The diameter of the bed should be {Dia} and Lenght {L}")






The specific adsorbent capacity is: 1288.5258294175405 cal/mol
150 SLM of helium = 0.4465 g/s
We need to adsorb 1.0735477487320106 kg of N2 to run 4 days
We need 6.760376251461024 kg of carbon for the bed
This relates to a volume of 0.542592407289706 cuft
The diameter of the bed should be 6.36 and Lenght 29.512998631582825


In [15]:
#L = 24 #in#
D = 3
R = 12 #L/D#
L = 36 #R*D
print(L)
cuin_to_cc =16.3871
g_to_kg = 1/1000
capacity_carbon_N2 = 300 #cc/kgC Approx #300

V = (((L**3) * math.pi)/(4 * R**2))*cuin_to_cc #cc
mass = V * .44*g_to_kg #kg
print(mass)
density_n2 = 794 #kg/m3

def get_adsorbed_carbon(rho,mass,capacity):
    m3_to_cm3 = 1/1000000
    g_to_kg = 1/1000
    adsorbed = (mass*rho * capacity * m3_to_cm3)/g_to_kg
    return adsorbed

adsorbed = get_adsorbed_carbon(density_n2,mass,capacity_carbon_N2)#kg
print(adsorbed)
break_through_time = adsorbed/(Mass_flow_rate_N2_JLab*day_to_sec)
print(break_through_time)




36
1.834803973580576
437.0503065068932
4.071084001835059


In [16]:
#Jlab code water

def polanyi_potential_theory(R,T,Ps,p):

    specific_adsorbent_capacity = R*T*math.log(Ps/p)
    return specific_adsorbent_capacity

def get_mass_carbon(rho,adsorbed,capacity):
    m3_to_cm3 = 1/1000000
    g_to_kg = 1/1000
    mass = (adsorbed*g_to_kg)* capacity
    return mass
def get_volume_carbon(mass):
    g_to_kg = 1/1000
    cm3_to_cuft = 3.53147e-5
    density_carbon = 1 #g/cc
    vol = mass/(density_carbon*g_to_kg)
    vol = vol*cm3_to_cuft
    return vol


def get_g_per_s_flow(SLM = 60):
    # Constants
    RHO_He_STP = 1         # helium density at STP [g/L]
    SECONDS_PER_MINUTE = 60

    # Convert SLM to g/s
    mass_flow_g_per_s = SLM * RHO_He_STP / SECONDS_PER_MINUTE

    print(f"{SLM} SLM of helium = {mass_flow_g_per_s:.4f} g/s")
    return mass_flow_g_per_s


#GIVENS
R = 0.2968 #kj/kg-k
T = 293 #K
Ps_H2O = 2.318 #kpa
ppm_H2O = 10 #pmm
P_total = 41.3685 #kpa

# #get partial_pressure N2
Partial_H2O = ppm_H2O*P_total/1e6

specific_adsorbent = polanyi_potential_theory(R,T,Ps_H2O,Partial_H2O)

#print(f"The specific adsorbent capacity is: {specific_adsorbent} kJ/kg")
KJ_to_cal = 239.006
kg_per_mol_H2O = 0.018015

specific_adsorbent_cal_per_mol = specific_adsorbent * KJ_to_cal * kg_per_mol_H2O


print(f"The specific adsorbent capacity is: {specific_adsorbent_cal_per_mol} cal/mol")


#Next find mass concentration of nitrogen in helium

mole_fraction_helium = 0.99
mol_he = 4.0026 #Kg/kmol
mass_helium_mix = mole_fraction_helium*mol_he #kg

mole_fraction_H2O_Jlab = ppm_H2O/1e6
mol_H2O = 18.01528 #Kg/kmol



mass_H2O_mix = mole_fraction_H2O_Jlab*mol_H2O #kg

Total_mass_mix = mass_helium_mix + mass_H2O_mix
percent_mass_H2O = mass_H2O_mix/Total_mass_mix

#Find Nitrogen Partial Flow
Mass_flow_rate_He_Jlab = get_g_per_s_flow() #g/s
Mass_flow_rate_H2O_JLab = Mass_flow_rate_He_Jlab*percent_mass_H2O #g/s

#Now we find the specific weight of adsorbent
break_through_time = 3 #Days
day_to_sec = 86400
H2O_needed_adsorbed = break_through_time*Mass_flow_rate_H2O_JLab*day_to_sec
print(f"We need to adsorb {H2O_needed_adsorbed/1000} kg of H2O to run {break_through_time} days")

capacity_13X_H2O = .50 
density_H2O = 1000 #kg/m3

mass_of_13X = get_mass_carbon(density_H2O,H2O_needed_adsorbed,capacity_13X_H2O)
print(f"We need {mass_of_13X} kg of 13X for the bed")
volume_13X = get_volume_carbon(mass_of_13X)
print(f"This relates to a volume of {volume_13X} cuft")

Dia = 3 #inch
L = (volume_13X * cuft_to_cuin)/((Dia/2)**2 * math.pi)

print(Dia, L)




The specific adsorbent capacity is: 3231.7760870523894 cal/mol
60 SLM of helium = 1.0000 g/s
We need to adsorb 0.011783624206665927 kg of H2O to run 3 days
We need 0.0058918121033329635 kg of 13X for the bed
This relates to a volume of 0.0002080675768855726 cuft
3 0.050864614438643506


In [52]:
#Determining the pressure drop of the Bed
#using Ergun Equation
Dia = 6*0.03 #m
density_he = .479*1000 #g/m3
dynamic_visocity = 9.6e-6 #microPa/s helium 100k
Area = math.pi*(Dia/2)**2
flow_speed = (Mass_flow_rate_He_Jlab/((density_he)*Area))
print(flow_speed)
porisity = .9#1/1.5e-7 #approx

Mod_Reynolds = density_he*flow_speed*Dia/((1-porisity)*dynamic_visocity)
f_p = (150/Mod_Reynolds) + 1.75
constant = porisity**3 / (1 - porisity)
constant2 = Dia / (density_he * flow_speed**2)

Delta_P_L = f_p/(constant2*constant)
print(Delta_P_L)

0.036631192603176
0.8572094692614869


In [54]:
#pressure drop in coil
#Darcy-Weisbach
flow_rate = .0025 #m3/s
dia = .01 #m 
Delta_P_L = (128/math.pi)*dynamic_visocity*flow_rate/( dia ** 4)
print(Delta_P_L)

97.7847970356605


In [56]:
flow_rate = 150 #SLM
mol_he = 0.0040026 # kg/mol
n_dot = flow_rate/(60*22.414) #mol/s
m_dot = n_dot*mol_he #kg/s
cp_he = 5251 #J/kgK

T_in = 293 #K
T_out = 80 #K

dT = T_in - T_out

Q = m_dot*cp_he*dT #Watts

L_n2 = 200e3 #J/kg
m_dot_LN2 = Q/L_n2

print(f"The flow rate of LN2 boil off from the heat exchanger is: {m_dot_LN2}. kg/s")

rho_n2 = .800

vol = m_dot_LN2/rho_n2
print(f"This is {vol} L/s")

vol = vol*61.0237*3600 #cuin/hr

Dia = 17 #inch
L = (vol)/((Dia/2)**2 * math.pi)

print(f"This is {L} in inches per hr")


The flow rate of LN2 boil off from the heat exchanger is: 0.0024966315716739534. kg/s
This is 0.0031207894645924415 L/s
This is 3.0204926561701106 in inches per hr


In [26]:
#How Long should coil be?
T_bath = 77 #k
LMTD = get_LMTD_counter(T_in,T_bath,T_out,T_bath)
print(f"The LMTD is {LMTD} K")

Do = .5
wall_thickness = .035
Di = Do - wall_thickness

The LMTD is 49.805150571119526 K


In [61]:
import math

# inputs
flow_SLM = 150.0
Tin = 293.0
Tout = 80.0
Tb = 77.0

in_to_m = 0.0254
Do = 5/8*in_to_m      # m (0.5 in)
t = 0.049*in_to_m       # m (0.035 in)
M_he = 4.0026e-3   # kg/mol
Vm_stp = 22.414    # L/mol
cp = 5193.0        # J/kgK

# compute mass flow and Q
n_dot = flow_SLM/(60.0*Vm_stp)   # mol/s
m_dot = n_dot * M_he             # kg/s
Q = m_dot * cp * (Tin - Tout)    # W

# mean properties approx
Tmean = (Tin + Tout)/2.0
Rspec = 8.314 / M_he
rho = 101325.0/(Rspec * Tmean)

A_flow = math.pi*(Do**2)/4.0
vel = m_dot/(rho*A_flow)

# property guesses (tweak as needed)
mu = 2.0e-5     # Pa.s (order)
k_g = 0.15      # W/mK (approx)
Pr = cp * mu / k_g

Re = rho * vel * Do / mu

# estimate internal h (Dittus-like; for low Re this is only an estimate)
Nu =  0.023*(Re**0.8)*(Pr**0.4)
h_i = Nu*(k_g/Do)

# external h (choose plausible boiling value)
h_o = 1000.0    # W/m2K (try 200, 500, 1500 to see sensitivity)

# wall conduction per area
k_wall = 167.0   # W/mK (AL)
R_wall = t/k_wall

U = 1/(1/h_i + R_wall + 1/h_o)

# LMTD
dT1 = Tin - Tb
dT2 = Tout - Tb
LMTD = (dT1 - dT2)/math.log(dT1/dT2)

A_needed = Q/(U * LMTD)
L_needed = A_needed/(math.pi * Do)

print(f"m_dot = {m_dot:.3e} kg/s, Q = {Q:.1f} W")
print(f"Re = {Re:.1f}, PR = {Pr}, Nu = {Nu}, h_i ~ {h_i:.1f} W/m2K, U ~ {U:.1f} W/m2K")
print(f"LMTD = {LMTD:.1f} K, Area = {A_needed:.4f} m2, Length = {L_needed:.2f} m")

#Finding the height of the coil
D = 17 #inch
pitch = 3 #in
L = L_needed*39.37 #in
print(f"We need {L} inches of tube")

Height = L*pitch/math.sqrt((math.pi * D)**2 + pitch**2)
print(f"The coild height should be {Height} inches")


m_dot = 4.464e-04 kg/s, Q = 493.8 W
Re = 1790.3, PR = 0.6924000000000001, Nu = 7.947198059216198, h_i ~ 75.1 W/m2K, U ~ 69.8 W/m2K
LMTD = 49.8 K, Area = 0.1420 m2, Length = 2.85 m
We need 112.11616695355113 inches of tube
The coild height should be 6.287914236436445 inches
